In [1]:
import json
import numpy as np
from numpy import log,sqrt,pi
import itertools
import os

## Define auxiliary functions

In [2]:
vev = 246.
yt = 172./(vev/np.sqrt(2))
yb = 4.7/(vev/np.sqrt(2))
yta = 1.77/(vev/np.sqrt(2))
d = 'dummyIndex' # define dummy index


def Delta(a,b):

    # Dummy indices are treated as strings,
    # so we need to deal with it separately.
    # Assume it always return 1
    
    if isinstance(a,int) and isinstance(b,int):
        if a == b:
            return 1
        else:
            return 0
    else:
        return 1

def evolve_gs(scale,alpha_s0 = 0.118):
    gs = np.sqrt(4 * np.pi * alpha_s0)
    # evolve gs from MZ to scale
    beta0 = 11 - 2 / 3 * 6
    return gs / np.sqrt(
        1 + 2 * beta0 * gs**2 / (4 * np.pi) ** 2 * np.log(scale / 91.1876)
    )


def computeCoeffs(funcDict,mST,mChi,g1,g2,g3,removeNonZero=True):
    all_indices = [1,2,3]
    results = []
    for coeff in funcDict:
        indices = funcDict[coeff]['indices']
        for index_values in itertools.product(*((all_indices for i in range(len(indices))))):
            coeff_0 = float(funcDict[coeff]['func'](*index_values,mST,mChi,0,0,g1,g2,g3)) # term independent of yDM
            coeff_2 = float(funcDict[coeff]['func'](*index_values,mST,mChi,1,0,g1,g2,g3) )- coeff_0 # term scaling as  yDM^2
            coeff_4 = float(funcDict[coeff]['func'](*index_values,mST,mChi,0,1,g1,g2,g3)) - coeff_0 # term scaling as  yDM^4
            index_str = str(index_values).replace(',','').strip()
            results.append([coeff,index_str,coeff_0,coeff_2,coeff_4])
    
    if removeNonZero:
        results = [tuple(r) for r in results[:] if sum(r[-3:-1]) != 0.0]
    
    results = np.array(results,dtype=[('coeff','<U10'),('indices','<U15'),('C0','<f8'),('C2','<f8'),('C4','<f8')])

    return results
    

def saveResults(results,mST,mChi,g1,g2,g3,outputFile=None):

    if outputFile is None:
        outputFile = f'coeff_values_mST_{mST:1.0f}_mChi_{mChi:1.0f}_test.csv'
    header = f'# SM parameters:\n#  g1 = {g1:1.2e}, g2 = {g2:1.2e}, g3 = {g3:1.2e}, yt = {yt:1.2e} , yb = {yb:1.2e}, yta = {yta:1.2e}'
    header += f'\n# UV parameters:\n#  mST = {mST:1.2f}, mChi = {mChi:1.2f}'
    header += '\n# The total value for the coefficient is given by C0 + yDM^2*C2 + yDM^4*C4, where yDM is the BSM coupling'
    header += '\nCoefficient,Indices,C0 (1/GeV^2),C2 (1/GeV^2),C4 (1/GeV^2)'
    np.savetxt(outputFile,results, comments='',header=header,
               fmt='%s,%s,%1.3e,%1.3e,%1.3e') 

def loadCoeffs(coeffsFile='coeffs_python.json'):

    with open(coeffsFile, 'r') as f:
        exprDict = json .load(f)

    funcDict = {}
    for coeff in exprDict:
        funcDict[coeff] = {'indices' : exprDict[coeff]['indices']}
        indices = ','.join(exprDict[coeff]['indices'])
        expr = exprDict[coeff]['expr']
        if indices.strip():
            exec(f"def {coeff}({indices},mST,mChi,yDM2,yDM4,g1,g2,g3): return {expr}")
        else:
            exec(f"def {coeff}(mST,mChi,yDM2,yDM4,g1,g2,g3): return {expr}")
        funcDict[coeff]['func'] = locals()[coeff]

    return funcDict

### SM parameters

In [4]:
aEW = 1.0/127.9
MZ = 91.1876
Gf = 1.166370e-05
aS = 0.130
MW = sqrt(MZ**2/2+sqrt(MZ**4/4-pi/sqrt(2)*aEW/Gf*MZ**2))
ee = np.sqrt(4*pi*aEW)
sw2 = 1.0 - (MW/MZ)**2 
cw = np.sqrt(1-sw2)
sw = np.sqrt(sw2)
g1 = 0.0
g2 = 0.0
g3 = np.sqrt(4*pi*aS)


## Load Coefficient Expressions

In [5]:
funcDict = loadCoeffs(coeffsFile='coeffs_python.json')

In [ ]:
mST = 1e4
mChi = 9e3
res = computeCoeffs(funcDict,mST,mChi,g1,g2,g3,removeNonZero=True)
for r in res:
    if r[0] != 'cuu':
        continue
    print(r)

('cuu', '(1 1 1 1)', -4.694444444444447e-13, 0.0, 0.0)
('cuu', '(1 1 2 2)', 2.347222222222223e-13, 0.0, 0.0)
('cuu', '(1 1 3 3)', 2.347222222222223e-13, -7.475919480988706e-13, 0.0)
('cuu', '(1 2 2 1)', -7.04166666666667e-13, 0.0, 0.0)
('cuu', '(1 3 3 1)', -7.04166666666667e-13, 0.0, 0.0)
('cuu', '(2 1 1 2)', -7.04166666666667e-13, 0.0, 0.0)
('cuu', '(2 2 1 1)', 2.347222222222223e-13, 0.0, 0.0)
('cuu', '(2 2 2 2)', -4.694444444444447e-13, 0.0, 0.0)
('cuu', '(2 2 3 3)', 2.347222222222223e-13, -7.475919480988706e-13, 0.0)
('cuu', '(2 3 3 2)', -7.04166666666667e-13, 0.0, 0.0)
('cuu', '(3 1 1 3)', -7.04166666666667e-13, 2.242775844296612e-12, 0.0)
('cuu', '(3 2 2 3)', -7.04166666666667e-13, 2.242775844296612e-12, 0.0)
('cuu', '(3 3 1 1)', 2.347222222222223e-13, 0.0, 0.0)
('cuu', '(3 3 2 2)', 2.347222222222223e-13, 0.0, 0.0)
('cuu', '(3 3 3 3)', -4.694444444444447e-13, 1.4951838961977415e-12, -2.9220056494838796e-12)
1.6732048802823662


## Loop over BSM Parameters

In [5]:
mSTlist = np.arange(200.,1500.,100.)
mChilist = np.arange(200.,1500.,50.)
npts = 0
resDir = '../../smefit/scan_files'
os.makedirs(resDir)
for (mST,mChi) in itertools.product(mSTlist,mChilist):
    if mChi >= mST:
        continue
    
    g3 = evolve_gs(scale=mST)
    outputFile = os.path.join(resDir,f'coeff_values_mST_{mST:1.0f}_mChi_{mChi:1.0f}.csv')
    results = computeCoeffs(funcDict,mST,mChi,g1,g2,g3,removeNonZero=True)
    saveResults(results,mST,mChi,g1,g2,g3,outputFile=outputFile)